In [2]:
import os
%pwd

'c:\\Users\\roypa\\Documents\\University of Manchester\\Text Mining\\Testing\\rp_text_summarizer\\research'

In [3]:
%pwd

os.chdir("../")
%pwd

'c:\\Users\\roypa\\Documents\\University of Manchester\\Text Mining\\Testing\\rp_text_summarizer'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    save_steps: int
    evaluation_strategy: str
    eval_steps: int
    gradient_accumulation_steps: int

In [5]:
from text_summarizer.constants import *
from text_summarizer.utils.common import read_yaml, create_directory

In [6]:
# configurations

class ConfigurationManager:
    def __init__(self, 
                 config_filepath=CONFIG_FILE_PATH, 
                 params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directory([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directory([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            model_ckpt = config.model_ckpt,
            num_train_epochs = params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            eval_steps=params.eval_steps,
            evaluation_strategy = params.evaluation_strategy,
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation_steps
        )

        return model_trainer_config

In [7]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch

c:\Users\roypa\anaconda3\conda\envs\textS\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-03-13 14:52:53,199: INFO: config: PyTorch version 2.2.0 available.]


In [8]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

        train_dataset_samsum_pt = load_from_disk(self.config.data_path)

        training_args = TrainingArguments(
            output_dir = self.config.root_dir,
            num_train_epochs = self.config.num_train_epochs,
            per_device_train_batch_size = self.config.per_device_train_batch_size,
            weight_decay = self.config.weight_decay,
            logging_steps = self.config.logging_steps,
            warmup_steps=self.config.warmup_steps,
            eval_steps=self.config.eval_steps,
            evaluation_strategy = self.config.evaluation_strategy,
            save_steps = self.config.save_steps,
            gradient_accumulation_steps = self.config.gradient_accumulation_steps
        )

        trainer = Trainer(
            model = model_pegasus,
            args = training_args,
            data_collator = seq2seq_data_collator,
            train_dataset = train_dataset_samsum_pt["test"],
            #eval_dataset= eval_dataset_samsum_pt["validation"]
        )

        trainer.train()
        
        # save model
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir, "pegasus-samsum-model"))

        # save tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir, "pegasus-samsum-tokenizer"))

In [9]:
%pip install transformers[torch]

Note: you may need to restart the kernel to use updated packages.


In [10]:
# data training pipeline
try:
    config = ConfigurationManager()
    data_trainer_config = config.get_model_trainer_config()
    data_trainer_config = ModelTrainer(config=data_trainer_config)
    data_trainer_config.train()
except Exception as e:
    raise e

[2024-03-13 14:52:56,591: INFO: common: yaml file config\config.yaml loaded successfully]
[2024-03-13 14:52:56,596: INFO: common: yaml file params.yaml loaded successfully]
[2024-03-13 14:52:56,597: INFO: common: directory artifacts created successfully]
[2024-03-13 14:52:56,598: INFO: common: directory artifacts/model_trainer created successfully]


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\roypa\anaconda3\conda\envs\textS\Lib\site-packages\accelerate\accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(
 20%|█▉        | 10/51 [01:44<07:04, 10.35s/it]

{'loss': 3.1612, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.2}


 39%|███▉      | 20/51 [03:16<04:50,  9.36s/it]

{'loss': 3.0344, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.39}


...

{'loss': 3.2559, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.78}


 98%|█████████▊| 50/51 [09:02<00:12, 12.09s/it]

{'loss': 2.7085, 'learning_rate': 5e-06, 'epoch': 0.98}


100%|██████████| 51/51 [09:13<00:00, 10.85s/it]
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8, 'forced_eos_token_id': 1}


{'train_runtime': 553.4438, 'train_samples_per_second': 1.48, 'train_steps_per_second': 0.092, 'train_loss': 3.0353969031689214, 'epoch': 1.0}
